# **Mobile Game Company's Rolling Retention Analysis**

In [1]:
#set GOOGLE_APPLICATION_CREDENTIALS=KEY_PATH
##keypath is the json key of service account
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=""

In [2]:
%load_ext google.cloud.bigquery

# Item Info table

In [3]:
%%bigquery
SELECT *
FROM
  `junobigqueryclass.game_data.item_info`
LIMIT
  1

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.34s/rows]


,item_id,price
0,35d02644f8f74e818546a6803600dd8a,2.0


# Matches Info table

In [4]:
%%bigquery
SELECT *
FROM `junobigqueryclass.game_data.matches_info` 
LIMIT 1

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.01rows/s]


,player_id,match_id,opponent_id,outcome,day
0,650cf321ff3e4dec8d247854d62f0604,a226a425199c4494bd2a5fca48d85ea9,e2cdc18efc534b4bb740d14aa45d5f7a,win,1


# Player Info table

In [5]:
%%bigquery
SELECT *
FROM `junobigqueryclass.game_data.player_info` 


Downloading: 100%|██████████████████████████████████████████████████████████| 49510/49510 [00:01<00:00, 28687.00rows/s]


,player_id,location,age,system,joined
0,49d65bd7b1bd4d398e4bb67fe6fb6082,Asia,8,Linux,104
1,eeae10da5c1747b38b51bf860399af2c,Africa,8,Linux,272
2,86478665e3194a52bc453be87c7760fc,South America,9,Linux,128
3,49dab7705da64b31b590ece570fcfdee,South America,9,Linux,148
4,cd7a16dfe302454cad858da1ebba52c9,North America,9,Linux,254
...,...,...,...,...,...
49505,1f5699cb2363477d8c72116c4685edb2,Oceania,31,Windows,100
49506,465033bf98d74d12bff6c01776cc791f,Oceania,32,Windows,8
49507,1329f92c6ce74f12b1c003892be4e132,Africa,32,Windows,116
49508,138b9537be514488baa2d8026f73807a,Asia,32,Windows,245


# Purchase Info table

In [6]:
%%bigquery
SELECT *
FROM `junobigqueryclass.game_data.purchase_info` 
LIMIT 1

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.14s/rows]


,player_id,item_id,day
0,a2850b5e0b054d0a97338c14d0fd8338,54c53d6ef25c4f94af77e48c75f7e802,1


## **Finding Player Joined and Retained players on a particular day**

#### 1. The total number of players who joined on a given day

In [7]:
%%bigquery
SELECT joined as joining_day, 
COUNT(player_id) players_joined
FROM `junobigqueryclass.game_data.player_info` 
GROUP BY 1
ORDER BY 1

Downloading: 100%|████████████████████████████████████████████████████████████████| 365/365 [00:00<00:00, 391.36rows/s]


,joining_day,players_joined
0,1,121
1,2,158
2,3,95
3,4,154
4,5,126
...,...,...
360,361,106
361,362,182
362,363,83
363,364,119


#### 2. Last play day table

In [8]:
%%bigquery
SELECT player_id as player_lastday,
    MAX(day) as lastplay_day
FROM `junobigqueryclass.game_data.matches_info` 
GROUP BY 1

Downloading: 100%|██████████████████████████████████████████████████████████| 49365/49365 [00:01<00:00, 30728.93rows/s]


,player_lastday,lastplay_day
0,650cf321ff3e4dec8d247854d62f0604,30
1,49483dcbeefe4f0fa0aade55fa0e7e54,33
2,4acd268a7717437cbcc172f215ba640c,46
3,74e7570d816442c891719a8296acab5a,33
4,1f360517123f47e79800b12a88a7d6ed,51
...,...,...
49360,da9c369c84194cb4873f624b07586011,365
49361,0c98196ad466432c9bc6aaf465b8cbee,365
49362,db4ed21e12e142aba00dadae824e4503,365
49363,77f5391cc83b41c7a55116b07c683f04,365


#### 3. Joining lastplay table for each player 

In [9]:
%%bigquery
SELECT players.player_id,
    players.joined as joining_day,
    lastplay.lastplay_day
FROM `junobigqueryclass.game_data.player_info` players
JOIN 
(
SELECT player_id as player_lastday,
    MAX(day) as lastplay_day
FROM `junobigqueryclass.game_data.matches_info` 
GROUP BY 1
) as lastplay

ON players.player_id = lastplay.player_lastday

Downloading: 100%|██████████████████████████████████████████████████████████| 49365/49365 [00:01<00:00, 33257.81rows/s]


,player_id,joining_day,lastplay_day
0,49d65bd7b1bd4d398e4bb67fe6fb6082,104,127
1,eeae10da5c1747b38b51bf860399af2c,272,314
2,86478665e3194a52bc453be87c7760fc,128,165
3,49dab7705da64b31b590ece570fcfdee,148,198
4,cd7a16dfe302454cad858da1ebba52c9,254,275
...,...,...,...
49360,1f5699cb2363477d8c72116c4685edb2,100,133
49361,465033bf98d74d12bff6c01776cc791f,8,57
49362,1329f92c6ce74f12b1c003892be4e132,116,139
49363,138b9537be514488baa2d8026f73807a,245,287


#### 4. Getting the list of retained player from above table
##### condition : if a given player play a match 30 days after he or she joined is a retained player

In [10]:
%%bigquery
SELECT days_info.joining_day,
        days_info.player_id as player_id,
        CASE 
        WHEN days_info.lastplay_day - days_info.joining_day > 30 THEN 1
        ELSE 0
        END AS retained
FROM
(SELECT players.player_id,
    players.joined as joining_day,
    lastplay.lastplay_day
FROM `junobigqueryclass.game_data.player_info` players
JOIN 
(
SELECT player_id as player_lastday,
    MAX(day) as lastplay_day
FROM `junobigqueryclass.game_data.matches_info` 
GROUP BY 1
) as lastplay

ON players.player_id = lastplay.player_lastday) as days_info
ORDER BY 1



Downloading: 100%|██████████████████████████████████████████████████████████| 49365/49365 [00:01<00:00, 32833.60rows/s]


,joining_day,player_id,retained
0,1,b7a8073b5d6d4da1b4123a94312d2da3,1
1,1,419611e6ec5a4679b8eaa0912f9dfbf1,0
2,1,7deb26015a5a4d7fa90f946b97c5be2a,1
3,1,cb0967d59c5c480d8342f0a20605ce5c,0
4,1,4a1f868490a74baaa245acb19a59133d,0
...,...,...,...
49360,365,ea8884d4f4d64a7eb17e0c87714cbef7,0
49361,365,08ba3a97b16048c3967c0d0be921a786,0
49362,365,257a0d154781439d844dc3fd371c2bec,0
49363,365,1ce90a5a4f994dc9b226cd9e85b1f818,0


#### 5.  Aggregating over Joining_Day to get total players joined vs total players retained

In [11]:
%%bigquery

SELECT DISTINCT joining_day as day, 
    COUNT(player_id) OVER (PARTITION BY joining_day) players_joined,
    SUM(retained) OVER (PARTITION BY joining_day) AS players_retained,

FROM
(
SELECT days_info.joining_day,
        days_info.player_id as player_id,
        CASE 
        WHEN days_info.lastplay_day - days_info.joining_day > 30 THEN 1
        ELSE 0
        END AS retained
FROM
(SELECT players.player_id,
    players.joined as joining_day,
    lastplay.lastplay_day
FROM `junobigqueryclass.game_data.player_info` players
JOIN 
(
SELECT player_id as player_lastday,
    MAX(day) as lastplay_day
FROM `junobigqueryclass.game_data.matches_info` 
GROUP BY 1
) as lastplay

ON players.player_id = lastplay.player_lastday) as days_info
ORDER BY 1
) AS t2
ORDER BY 1


Downloading: 100%|████████████████████████████████████████████████████████████████| 365/365 [00:01<00:00, 332.12rows/s]


,day,players_joined,players_retained
0,1,121,82
1,2,158,105
2,3,95,62
3,4,154,99
4,5,126,84
...,...,...,...
360,361,103,0
361,362,164,0
362,363,73,0
363,364,89,0


# Fractional Retention
#### *(players_retained divided by players_joined)*

#### 6. Dividing column third by second to get fractional retention

In [12]:
%%bigquery
SELECT 
  DISTINCT joining_day as day, 
  COUNT(player_id) OVER (PARTITION BY joining_day) players_joined, 
  SUM(retained) OVER (PARTITION BY joining_day) AS players_retained, 
  SUM(retained) OVER (PARTITION BY joining_day) / COUNT(player_id) OVER (PARTITION BY joining_day) fractional_retention 
FROM 
  (
    SELECT 
      days_info.joining_day, 
      days_info.player_id as player_id, 
      CASE WHEN days_info.lastplay_day - days_info.joining_day > 30 THEN 1 ELSE 0 END AS retained 
    FROM 
      (
        SELECT 
          players.player_id, 
          players.joined as joining_day, 
          lastplay.lastplay_day 
        FROM 
          `junobigqueryclass.game_data.player_info` players 
          JOIN (
            SELECT 
              player_id as player_lastday, 
              MAX(day) as lastplay_day 
            FROM 
              `junobigqueryclass.game_data.matches_info` 
            GROUP BY 
              1
          ) as lastplay ON players.player_id = lastplay.player_lastday
      ) as days_info
  ) 
ORDER BY 
  1


Downloading: 100%|████████████████████████████████████████████████████████████████| 365/365 [00:00<00:00, 378.97rows/s]


,day,players_joined,players_retained,fractional_retention
0,1,121,82,0.677686
1,2,158,105,0.664557
2,3,95,62,0.652632
3,4,154,99,0.642857
4,5,126,84,0.666667
...,...,...,...,...
360,361,103,0,0.000000
361,362,164,0,0.000000
362,363,73,0,0.000000
363,364,89,0,0.000000


# Do players with rolling 30-day retention spend more?


#### 7. Joining Purchase_info table and item_info table to get money spent by each player

In [13]:
%%bigquery
SELECT player_id,
       SUM(price)
FROM `junobigqueryclass.game_data.item_info` i
JOIN `junobigqueryclass.game_data.purchase_info` p
ON i.item_id = p.item_id
GROUP BY 1

Downloading: 100%|██████████████████████████████████████████████████████████| 49510/49510 [00:01<00:00, 41680.06rows/s]


,player_id,f0_
0,a2850b5e0b054d0a97338c14d0fd8338,1578.55
1,0240b331df71437c91112b8937489c7a,1161.03
2,1f360517123f47e79800b12a88a7d6ed,1335.42
3,d1c44236ef404610b50664f01f3fc6c1,1020.13
4,98f3e612de604b1b957cc099eaa8627e,984.37
...,...,...
49505,52eac1671cb84c5f979c6ec1768398e0,1104.67
49506,616324200217411bb101d851d4596c5c,1220.26
49507,a019ea71589246ca8bfdfd6fcbe627e9,964.59
49508,914813926fcc411493a2a45a8dc14b57,1034.18


#### 8.  Using the same logic from cell In[32] above to get retained and unretained players and joining with the table above

In [14]:
%%bigquery
SELECT *
FROM
(SELECT player_id as pid,
       SUM(price) as amount_spent
FROM `junobigqueryclass.game_data.item_info` i
JOIN `junobigqueryclass.game_data.purchase_info` p
ON i.item_id = p.item_id
GROUP BY 1) AS player_spent

JOIN
(SELECT days_info.player_id as player_id,
        CASE 
        WHEN days_info.lastplay_day - days_info.joining_day > 30 THEN "retained"
        ELSE 'unretained'
        END AS player_status
FROM
(SELECT players.player_id,
    players.joined as joining_day,
    lastplay.lastplay_day
FROM `junobigqueryclass.game_data.player_info` players
JOIN 
(
SELECT player_id as player_lastday,
    MAX(day) as lastplay_day
FROM `junobigqueryclass.game_data.matches_info` 
GROUP BY 1
) as lastplay

ON players.player_id = lastplay.player_lastday) as days_info) as player_status

ON player_status.player_id = player_spent.pid

Downloading: 100%|██████████████████████████████████████████████████████████| 49365/49365 [00:01<00:00, 25745.70rows/s]


,pid,amount_spent,player_id,player_status
0,eeae10da5c1747b38b51bf860399af2c,1163.67,eeae10da5c1747b38b51bf860399af2c,retained
1,86478665e3194a52bc453be87c7760fc,1100.68,86478665e3194a52bc453be87c7760fc,retained
2,49dab7705da64b31b590ece570fcfdee,1382.36,49dab7705da64b31b590ece570fcfdee,retained
3,34c91fdb23004044848f9c05fa38fd31,1378.18,34c91fdb23004044848f9c05fa38fd31,retained
4,327b96a94ed54b8cabbc1d15ce3f0342,1401.27,327b96a94ed54b8cabbc1d15ce3f0342,retained
...,...,...,...,...
49360,47aa779fd31640b5a0df133c66b08d5d,860.92,47aa779fd31640b5a0df133c66b08d5d,unretained
49361,dc318ed15f614b47bd1df38cdc2813d5,1202.03,dc318ed15f614b47bd1df38cdc2813d5,unretained
49362,592c04c752b1496e86f50f40ee4b8809,940.27,592c04c752b1496e86f50f40ee4b8809,unretained
49363,4ae3240eb796467baadf470a0709a639,650.45,4ae3240eb796467baadf470a0709a639,unretained


#### 9. Finally we aggregate the data over the `player_status` to find the sum of the amount spent by retained and unretained players and compare it.

In [15]:
%%bigquery
SELECT player_status,
    SUM(amount_spent) as total_spent
FROM

(SELECT *
FROM
(SELECT player_id as pid,
       SUM(price) as amount_spent
FROM `junobigqueryclass.game_data.item_info` i
JOIN `junobigqueryclass.game_data.purchase_info` p
ON i.item_id = p.item_id
GROUP BY 1) AS player_spent

JOIN
(SELECT days_info.player_id as player_id,
        CASE 
        WHEN days_info.lastplay_day - days_info.joining_day > 30 THEN "retained"
        ELSE 'unretained'
        END AS player_status
FROM
(SELECT players.player_id,
    players.joined as joining_day,
    lastplay.lastplay_day
FROM `junobigqueryclass.game_data.player_info` players
JOIN 
(
SELECT player_id as player_lastday,
    MAX(day) as lastplay_day
FROM `junobigqueryclass.game_data.matches_info` 
GROUP BY 1
) as lastplay

ON players.player_id = lastplay.player_lastday) as days_info) as player_status

ON player_status.player_id = player_spent.pid) as table1

GROUP BY 1
ORDER BY 2 DESC

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.02rows/s]


,player_status,total_spent
0,retained,40460972.40
1,unretained,18501435.71


#### *This shows the retained players have spent more than the unretained players*

## ------------------------------------------------------------------

# **Do players with rolling 30-day retention come from specific regions?**

#### 10. Using the query in In[32] filtering the data for retained players and joining it with player info table to get location specific retention data for players.

In [16]:
%%bigquery
SELECT location,
    COUNT(player_id) as total_retained_players
FROM
(
SELECT days_info.joining_day,
        days_info.player_id as pid
    
FROM
(SELECT players.player_id,
    players.joined as joining_day,
    lastplay.lastplay_day
FROM `junobigqueryclass.game_data.player_info` players
JOIN 
(
SELECT player_id as player_lastday,
    MAX(day) as lastplay_day
FROM `junobigqueryclass.game_data.matches_info` 
GROUP BY 1
) as lastplay

ON players.player_id = lastplay.player_lastday) as days_info

WHERE days_info.lastplay_day - days_info.joining_day > 30
) AS r_info

JOIN `junobigqueryclass.game_data.player_info` as p_info

ON p_info.player_id = r_info.pid
GROUP BY 1
ORDER BY 2 DESC

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  5.04rows/s]


,location,total_retained_players
0,South America,5296
1,Africa,5242
2,North America,5160
3,Asia,5124
4,Europe,5113
5,Oceania,5047


###  *Most Retained players came from South America*